# Lesson 2: Gradient Descent and Optimization

Intro to autograd, GD, a typical optimization loop

Borrow fairly heavily from AIAIART #1

GD from scratch

Improvements in optimization (momentum, adaptive methods, maybe even a bonus section on second order stuff like shampoo and a paper read of LocoProp)

### Exercise: optimize pixels in an image to match some loss (without explaining it)? OR have a black box function that goes from z -> orb, craft a loss for orb color and optimize z to get a particular orb color. Later in the GAN section we'll make the GAN which will hide behind the scenes and enable this!